In [ ]:
import pandas as pd
import numpy as np
import os
import glob
from glob import glob
import re
import luigi
from operator import itemgetter
from persiantools import characters
from persiantools.jdatetime import JalaliDate
import persiantools
import datetime
from re import search
import itertools
from statistics import mode
import datetime as dt
from joblib import Parallel, delayed
import chardet
import requests
from bs4 import BeautifulSoup
import json
import time
import io
import jdatetime
import TSETMCScraping
import ar_to_fa
import clean_cols
import str_to_date
import str_to_jdate
import str_to_time
import cleaning_path
from luigi.mock import MockTarget
import stock_information_cleaning

In [ ]:
class TSETMC_Sraping(luigi.Task):
    read_path= luigi.Parameter(default=None)
    dirr_path= luigi.Parameter(default=None)
    def requiers(self):
        return None
    def output(self):
        return luigi.LocalTarget(self.dirr_path)
    # def output(self):
    #     return MockTarget("first task", mirror_on_stderr=True)
    def run(self):
        Data_scraping= input('wich data do you want to scrap: (trade\information\shareholders) ')
        if Data_scraping =='information':
            if os.path.exists(self.dirr_path):
                pass
            else:
                scrap_links= pd.read_excel(self.read_path)
                df= TSETMCScraping.stock_information(scrap_links)
                df= cleaning_path.column_clustering(df)
                df= stock_information_cleaning.string_clean(df)
                df= stock_information_cleaning.price_clean(df)
                df= stock_information_cleaning.code_clean(df)
                df.to_csv(self.output().path, index=False, encoding='utf-8-sig')
        if Data_scraping == 'trade':
            if os.path.exists(self.dirr_path):
                pass
            else:
                scrap_links= pd.read_excel(self.read_path)
                df= TSETMCScraping.trade_history(scrap_links)
                df= cleaning_path.clean_cols(df)
                df= cleaning_path.ar_to_fa(df)
                df= cleaning_path.str_to_date(df)
                df= cleaning_path.date_to_jdate(df)
                df.to_csv(self.output().path, index=False, encoding='utf-8-sig')
        if Data_scraping == 'shareholders':
            if os.path.exists(self.dirr_path):
                pass
            else:
                scrap_links= pd.read_excel(self.read_path)
                df= TSETMCScraping.shareholders(scrap_links)
                df.to_csv(self.output().path, index=False, encoding='utf-8-sig')

Runnnig Luigi

In [ ]:
if __name__=='__main__':
    luigi.build([TSETMC_Sraping(read_path="C:\\Users\\ah.roudsaz\\Desktop\\test.xlsx",
                dirr_path="C:\\Users\\ah.roudsaz\\Desktop\\output.csv")],local_scheduler=True)